In [32]:
import pickle
import numpy as np
import os
import os.path as osp
import sys
import mxnet as mx
import cv2

In [39]:
class RecBuilder():
    def __init__(self, path, image_size=(112, 112)):
        self.path = path
        self.image_size = image_size
        self.last_label = -1
        self.widx = 1
        if not os.path.exists(path):
            os.makedirs(path)
        self.writer = mx.recordio.MXIndexedRecordIO(
            os.path.join(path, 'vgg.idx'), os.path.join(path, 'vgg.rec'),
            'w')
        self.label_stat = [-1, -1]
        self.identities = []

    def add(self, label, imgs, imgpath):
        #img should be BGR
        assert label >= 0
        assert label > self.last_label
        assert len(imgs) > 0
        idflag = [self.widx, -1]
        for imga in imgs:
            if imga=='.ipynb_checkpoints':
                continue
            img = cv2.imread(imgpath+imga)
            idx = self.widx
            self.widx += 1
            header = mx.recordio.IRHeader(0, label, idx, 0)
            if isinstance(img, np.ndarray):
                s = mx.recordio.pack_img(header,
                                         img,
                                         quality=95,
                                         img_fmt='.png')
            else:
                s = mx.recordio.pack(header, img)
            self.writer.write_idx(idx, s)
        idflag[1] = self.widx
        self.identities.append(idflag)
        if self.label_stat[0] < 0:
            self.label_stat = [label, label]
        else:
            self.label_stat[0] = min(self.label_stat[0], label)
            self.label_stat[1] = max(self.label_stat[1], label)
        self.last_label = label

    def close(self):
        id_idx = self.widx
        for id_flag in self.identities:
            idx = self.widx
            self.widx += 1
            _header = mx.recordio.IRHeader(0, id_flag, idx, 0)
            s = mx.recordio.pack(_header, b'')
            self.writer.write_idx(idx, s)

        print('id0:', (id_idx, self.widx))
        idx = 0
        _header = mx.recordio.IRHeader(0, (id_idx, self.widx), idx, 1)
        s = mx.recordio.pack(_header, b'')
        self.writer.write_idx(0, s)
        print('label stat:', self.label_stat)

        with open(os.path.join(self.path, 'property'), 'w') as f:
            f.write("%d,%d,%d\n" % (self.label_stat[1] + 1, self.image_size[0],
                                    self.image_size[1]))

In [44]:
cls = RecBuilder('/data/notebook/')

In [47]:
import cv2

path = '/data/notebook/img/'

lst = os.listdir(path)
label = 0
lst.sort()
for i in lst:
    if i == '.DS_Store' or i == '.ipynb_checkpoints':
        continue
    if label%50==0:
        print(label)

    imglst = os.listdir(path+i)
    cls.add(label,imglst,path+i+'/')
    label+=1

cls.close()
print(label)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
id0: (7141, 9249)
label stat: [0, 2107]
2108
